In [ ]:
from collections import namedtuple
import dataclasses as dc
import pathlib
import pprint
import random
import uuid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pendulum
import seaborn as sns

In [ ]:
from orchid import (project_store as ops)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet
# noinspection PyUnresolvedReferences
from System import DateTime

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), 
                                  project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: ops.ProjectStore(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
def get_project_monitors(proj):
    return proj.Monitors.Items

project_monitors = toolz.pipe(
    projects,
    toolz.valmap(get_project_monitors),
)
project_monitors

In [ ]:
toolz.pipe(project_monitors.values(),
           toolz.map(len),
           list,)

In [ ]:
@dc.dataclass
class MonitorBasics:
    object_id: uuid.UUID
    name: str
    display_name: str
    start_time: pendulum.DateTime
    stop_time: pendulum.DateTime

@toolz.curry
def summarize_monitor_basics(m):
    result = MonitorBasics(m.ObjectId, m.DisplayName, m.Name, 
                           m.StartTime.ToString('o'), m.StopTime.ToString('o'))
    return result

project_monitor_summaries = toolz.pipe(
    project_monitors,
    toolz.valmap(toolz.map(summarize_monitor_basics)),
    toolz.valmap(list),
)
project_monitor_summaries

In [ ]:
selected_field_name = 'bakken'

In [ ]:
def make_id_summary(field_name, summaries):
    field_name_summaries = summaries[field_name]
    return {
        'field': field_name,
        'Object Id': [s.object_id for s in field_name_summaries],
        # 'Display Name': [s.display_name for s in field_name_summaries],
        # 'Name': [s.name for s in field_name_summaries],
        'Start Time': [s.start_time for s in field_name_summaries],
        'Stop Time': [s.stop_time for s in field_name_summaries],
    }

In [ ]:
pd.DataFrame(data=make_id_summary('bakken', project_monitor_summaries))

In [ ]:
pd.DataFrame(data=make_id_summary('montney', project_monitor_summaries))

In [ ]:
@dc.dataclass
class PyTick:
    sample_at: DateTime
    sample_value: float
        
MonitorSummary = namedtuple('MonitorSummary', ['object_id',
                                               'name', 
                                               'display_name',
                                               'stage_no',
                                               'start',
                                               'stop',
                                               'time_series_id',
                                               'quantity_name',
                                               'sample_count',
                                               'samples'])
        
def monitors_time_series(ms):
    result = toolz.pipe(
        ms,
        toolz.map(lambda m:  MonitorSummary(
            object_id=m.ObjectId.ToString(), 
            name=m.Name, 
            display_name=m.DisplayName,
            stage_no=m.Stage.DisplayStageNumber if 'Stage' in m.GetType().Name else 'N/A',
            start=m.StartTime.ToString('s'),
            stop=m.StopTime.ToString('s'),
            time_series_id=str(m.TimeSeries.ObjectId), 
            quantity_name=m.TimeSeries.SampledQuantityName,
            sample_count=len(m.TimeSeries.GetOrderedTimeSeriesHistory()), 
            samples=m.TimeSeries.GetOrderedTimeSeriesHistory())),
        lambda summary: sorted(summary, key=lambda ms: ms.display_name),
        list,
    )
    return result

project_monitor_time_series = toolz.pipe(
    project_monitors,
    toolz.valmap(monitors_time_series),
    toolz.valmap(toolz.map(lambda ms: {ms.display_name: ms})),
    toolz.valmap(toolz.merge),
)
pprint.pprint(project_monitor_time_series)

In [ ]:
def shuffle_em(es):
    random.shuffle(es)
    return es

unshuffled_sample_stages = {
    'Demo_1H': [8, 22, 36, 50] * 2,
    'Demo_2H': [1, 14, 29, 43] * 2,
    'Demo_4H': [6, 15, 25, 35] * 2,
}
sample_stages = toolz.merge(
    toolz.valmap(shuffle_em, unshuffled_sample_stages),
    {'Hori_02': [1] * 8,
     'Vert_01': [1] * 8}
)
sample_stages

In [ ]:
sample_stages = {
    'Demo_1H': [22, 50, 22, 8, 36, 50, 36, 8],
    'Demo_2H': [29, 14, 1, 29, 1, 14, 43, 43],
    'Demo_3H': ['MonitorWell'] * 8,
    'Demo_4H': [35, 25, 6, 35, 15, 15, 25, 6],
    'Hori_02': [1] * 8,
    'Vert_01': [1] * 8,
}

In [ ]:
sample_indices = {
    'Demo_1H': [0, 6078, 24489, 35902, 47331, 56567, 94551, 114594],
    'Demo_2H': [0, 1105, 14475, 18418, 33263, 83966, 87110, 114594],
    'Demo_3H': [0, 9872, 38744, 47718, 58806, 58869, 62026, 62026],
    'Demo_4H': [0, 9872, 38744, 47718, 58806, 58869, 62026, 62757],
    'Hori_02': [0, 14848, 67025, 138193, 196604, 252497, 256716, 325041],
    'Vert_01': [0, 6652, 90175, 134841, 211515, 222007, 300671, 322124],
}

In [ ]:
toolz.pipe(
    project_monitor_time_series,
    # toolz.get_in(['bakken', 'Demo_1H - stage 50']),
    # toolz.get_in(['bakken', 'Demo_2H - stage 1']),
    # toolz.get_in(['bakken', 'Demo_3H - MonitorWell']),
    # toolz.get_in(['bakken', 'Demo_4H - stage 25']),
    toolz.get_in(['montney', 'Hori_02 - 0 - stage 1']),
    # lambda ms: toolz.get(sample_indices['Demo_1H'], ms.samples),
    # lambda ms: toolz.get(sample_indices['Demo_2H'], ms.samples),
    # lambda ms: toolz.get(sample_indices['Demo_3H'], ms.samples),
    # lambda ms: toolz.get(sample_indices['Demo_4H'], ms.samples),
    lambda ms: toolz.get(sample_indices['Hori_02'], ms.samples),
    toolz.map(lambda tick: PyTick(sample_at=tick.Timestamp.ToString('o'), 
                                  sample_value=tick.Value)),
    list,
)

In [ ]:
def sample_monitor_summary(monitor_summary):
    result = toolz.assoc(
        monitor_summary,
        'samples',
        toolz.pipe(
            toolz.get(sample_indices[monitor_summary['name']], monitor_summary['samples']),
            toolz.map(lambda tick: PyTick(sample_at=tick.Timestamp.ToString('o'), 
                                  sample_value=tick.Value)),
            toolz.map(lambda pytick: {'time': pytick.sample_at, 'value': pytick.sample_value}),
            lambda tick_map: zip(sample_indices[monitor_summary['name']], tick_map),
            toolz.map(lambda t: toolz.assoc(t[1], 'index', t[0])),
            list,
       )
    )
    return result
        

project_monitor_samples = toolz.pipe(
    project_monitor_time_series,
    toolz.valmap(toolz.valmap(lambda ms: ms._asdict())),
    toolz.valmap(toolz.valmap(sample_monitor_summary)),
    toolz.valmap(toolz.valmap(toolz.keyfilter(lambda k: k == 'quantity_name' or k == 'samples'))),
)
project_monitor_samples